In [1]:
import cv2
import numpy as np
import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.activations import *
from keras.optimizers import Adam,RMSprop,SGD
x = np.load("x.npy")
y = np.load("y.npy")
shuffe_index = np.random.permutation(x.shape[0])
N = x.shape[0]
K = int(0.8*N)#number of train samples
index = np.random.permutation(N)
x_train = x[index[0:K]]
y_train = y[index[0:K]]
x_test = x[index[K:]]
y_test = y[index[K:]]
print(N, K, N - K)
print(x.shape)

Using TensorFlow backend.
c:\python 3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python 3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python 3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python 3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1

304 243 61
(304, 64, 64, 3)


In [2]:
print(x_train)



[[[[ 85 118 114]
   [ 84 119 115]
   [ 89 125 125]
   ...
   [ 43  63  58]
   [ 45  62  58]
   [ 45  62  58]]

  [[ 83 116 112]
   [ 83 118 114]
   [ 86 125 123]
   ...
   [ 43  63  58]
   [ 46  63  59]
   [ 46  63  59]]

  [[ 82 118 112]
   [ 81 118 114]
   [ 81 121 119]
   ...
   [ 43  63  58]
   [ 45  62  58]
   [ 44  61  57]]

  ...

  [[ 98 140 145]
   [ 97 139 144]
   [ 95 138 141]
   ...
   [126 156 157]
   [107 134 138]
   [101 128 132]]

  [[ 81 121 126]
   [ 80 120 125]
   [ 77 117 122]
   ...
   [140 169 173]
   [133 160 164]
   [130 157 161]]

  [[ 77 117 122]
   [ 76 116 121]
   [ 73 113 118]
   ...
   [140 171 174]
   [137 164 168]
   [132 159 163]]]


 [[[ 56  93  85]
   [ 56  93  85]
   [ 65  99  92]
   ...
   [ 39  60  57]
   [ 38  57  54]
   [ 38  57  54]]

  [[ 54  91  83]
   [ 55  92  84]
   [ 63  97  90]
   ...
   [ 39  60  57]
   [ 38  57  54]
   [ 38  57  54]]

  [[ 47  83  77]
   [ 48  84  78]
   [ 56  92  86]
   ...
   [ 41  61  56]
   [ 42  59  55]
   [ 44  61

In [3]:
model = Sequential()

model.add(Conv2D(8, kernel_size=(5, 5), strides= (2, 2), padding='same',input_shape=( 64, 64, 3,)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, kernel_size=(3, 3), strides= (1, 1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), strides= (1, 1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(2))
model.add(Activation('sigmoid'))
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy',optimizer=optimizer)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 8)         608       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 16)        1168      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 16)          0         
________________________________________________

In [4]:
print (x_test.shape[0], x_train.shape[0])

61 243


In [5]:
def generator(x, y, batch_size):
    samples = x.shape[0]
    steps = samples/batch_size
    counter =0
    while True:
        batch_x = np.array(x[batch_size*counter: batch_size*(counter+1)]).astype(np.float32)
        batch_x = batch_x/255.0
        batch_y = np.array(y[batch_size*counter: batch_size*(counter+1)]).astype(np.float32)
        counter+=1
        yield batch_x, batch_y
        if counter >= steps:
            counter = 0
# batch_size = 128
batch_size = 32
epochs=50
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("models/sign.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history= model.fit_generator(generator(x_train, y_train, batch_size), epochs=epochs, steps_per_epoch = x_train.shape[0]/batch_size, 
                             validation_data=generator(x_test, y_test, batch_size), validation_steps = x_test.shape[0]/batch_size, callbacks=[checkpoint])


Epoch 1/50
8/7 [===============================] - 0s 48ms/step - loss: 0.4926 - val_loss: 0.3511

Epoch 00001: val_loss improved from inf to 0.35113, saving model to models/sign.h5
Epoch 2/50
8/7 [===============================] - 0s 14ms/step - loss: 0.3268 - val_loss: 0.2611

Epoch 00002: val_loss improved from 0.35113 to 0.26112, saving model to models/sign.h5
Epoch 3/50
8/7 [===============================] - 0s 12ms/step - loss: 0.2668 - val_loss: 0.2434

Epoch 00003: val_loss improved from 0.26112 to 0.24335, saving model to models/sign.h5
Epoch 4/50
8/7 [===============================] - 0s 12ms/step - loss: 0.2669 - val_loss: 0.1695

Epoch 00004: val_loss improved from 0.24335 to 0.16945, saving model to models/sign.h5
Epoch 5/50
8/7 [===============================] - 0s 11ms/step - loss: 0.2338 - val_loss: 0.1265

Epoch 00005: val_loss improved from 0.16945 to 0.12653, saving model to models/sign.h5
Epoch 6/50
8/7 [===============================] - 0s 12ms/step - loss: 0

In [5]:
import cv2
import numpy as np
from keras.models import load_model
import time
model = load_model("models/sign.h5")


Instructions for updating:
Use tf.cast instead.


In [6]:
t = time.time()
predict = model.predict(x_test/255.0)
print(time.time()-t)
for i in range(x_test.shape[0]):
    img = x_test[i]
    img = cv2.resize(img, (128, 128))
    print(predict[i])
    if predict[i,1]>0.5:
        if predict[i,0]<0.9:
            cv2.rectangle(img, (0,0), (30, 30), (0, 0, 255), -1)
        else:
            cv2.rectangle(img, (98, 0), (128, 30), (0, 0, 255), -1)
    cv2.imshow("img", img)
    k =cv2.waitKey(0)
    if k == ord('q'):
        break
cv2.destroyAllWindows()

4.329359292984009
[0.99999905 0.9999997 ]
[0.999925 1.      ]
[0.9999994 1.       ]
[0.00102904 1.        ]
[0.99999917 0.9999999 ]
[0.99778086 0.9998503 ]
[1.3351440e-05 9.9999976e-01]
[1.2129545e-05 9.9999994e-01]
[8.374453e-06 9.999996e-01]
[5.6919456e-04 1.0000000e+00]
[0.99999946 1.        ]
[0.99999976 1.        ]
[0.00371718 0.99999917]
[4.4497848e-04 9.9999368e-01]
[0.99999577 1.        ]
[1.14142895e-05 9.99999762e-01]
[0.9999995  0.99999994]
[2.233684e-04 1.000000e+00]
[0.9999914 1.       ]
[9.834766e-06 9.999999e-01]
[1.4662743e-05 9.9999988e-01]
[1.385808e-05 9.999998e-01]
[0.99998176 0.99999356]
[3.9523840e-04 9.9999887e-01]
[0.9999776 0.9999989]
[2.3198128e-04 9.9999988e-01]
[0.99996716 0.9999995 ]
[0.00242323 1.        ]
[0.99999857 1.        ]
[4.3100119e-04 9.9999964e-01]
[0.99997735 0.99998635]
[0.9999994 0.9999995]
[7.480383e-06 9.999997e-01]
[0.99999964 0.9999999 ]
[0.00110182 0.99999917]
[1.4644861e-04 1.0000000e+00]
[7.605553e-05 9.999994e-01]
[0.999716   0.999999

In [7]:
print (x_test.shape)

(61, 64, 64, 3)


In [45]:
def classify():
    img = cv2.imread('test/cropleft1.png')

    img = cv2.resize(img, (64, 64))       
#     print img.shape
#     with graph.as_default():
#         set_session(sess)
#         side, conf = model.predict(img)
    side, conf = model.predict(np.array([img])/255.0)[0]
    

    print side, conf


In [46]:
classify()

5.090714e-06 1.0


In [3]:
import timeit
from datetime import timedelta

# sess = tf.Session()

# graph = tf.get_default_graph()

# set_session(sess)
# my_model = load_model('models/sign_fix.h5')
# my_model = load_model('models/sign_fix.h5')

sign_cascade = cv2.CascadeClassifier('models/cascade2.xml')

crop = np.zeros((64, 64), np.uint8)

def FindSign(img):
    signs = sign_cascade.detectMultiScale(img,
                                          scaleFactor=1.03,
                                          minNeighbors=2,
                                          minSize=(19, 19),
                                          maxSize=(60, 60),
                                          flags=0)
    return signs


def Cascade_sign(img_):
    global crop, sess, graph
    t = 2
    side = None
    img = img_[0:100, ...].copy()
    img2 = img_[0:100, ...].copy()
    signs = FindSign(img2)
    position = None
    for (x, y, w, h) in signs:
        if x > 5 and y > 5 and x + w < 475 and y + h < 95 and 1.0 * h / w > 0.85 and 1.0 * h / w < 1.3:
            print(w, h)
            position = [x, y, w, h]
            crop = img[y - t:y + h + t, x - t:x + w + t, :]
            crop = cv2.resize(crop, (64, 64))
            start = timeit.default_timer()
#             with graph.as_default():
#                 set_session(sess)
#                 _side, confident = my_model.predict(np.array([crop]) /
#                                                     255.0)[0]
#                 print('toi dang o day')
#                 print('confident = ', confident)
            _side, confident = model.predict(np.array([crop]) /
                                                    255.0)[0]
            stop = timeit.default_timer()
            print ('Thoi gian chay model la ', stop - start)
            #_side, confident = sign_model.predict(np.array([crop])/255.0)[0]
            # _side = 0
            # confident = 0.9
            if _side < 0.5:
                _side = 0
            else:
                _side = 1
            if confident > 0.5:
                side = _side
            # cv2.rectangle(img,(x-t,y-t),(x+w+t,y+h+t),(0,255,0),2)
            # font = cv2.FONT_HERSHEY_SIMPLEX
            # cv2.putText(img,str(confident),(x-10,y-10), font, 0.5, (255,255,255), 1, cv2.LINE_AA)
            # #cv2.imshow('crop', crop)
    return crop, side, img, position

# time.sleep(5)


def testImage():
    img = cv2.imread('test/left1.png')

    crop1, side, img2, pos = Cascade_sign(img)
    #cv2.imshow('crop', crop)
    #cv2.imshow('sign', img2)
    #cv2.imshow('image', img)
    print ('side= ', side)
    print ('pos= ', pos)
    #getMask(img)

    cv2.waitKey(0)


In [ ]:
testImage()

In [44]:
img = cv2.imread('test/left1.png')

signs = FindSign(img)

for (x, y, w, h) in signs:
        if x > 5 and y > 5 and x + w < 475 and y + h < 95 and 1.0 * h / w > 0.85 and 1.0 * h / w < 1.3:
            print(w, h)

(26, 26)


In [ ]:
cl